In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern, WhiteKernel
from sklearn import linear_model
from sklearn.metrics import log_loss

from scipy.stats import zscore, norm

import itertools

np.random.seed(1)

# Load data (training and validation)
train_raw_teamstats = pd.read_excel('cts.xlsx', sheet_name='2017-2018')
train_raw_games = pd.read_csv('clean_2017-2018_combo.csv')
val_raw_teamstats = pd.read_excel('cts.xlsx', sheet_name='2016-2017')
val_raw_games = pd.read_csv('clean_2016-2017_combo.csv')

# Clean all the bloody whitespace
train_raw_teamstats = train_raw_teamstats.rename(columns=lambda x: x.strip())
train_raw_games = train_raw_games.rename(columns=lambda x: x.strip())
val_raw_teamstats = val_raw_teamstats.rename(columns=lambda x: x.strip())
val_raw_games = val_raw_games.rename(columns=lambda x: x.strip())

# Get the list of all stats
VARS = list(train_raw_teamstats)
VARS.remove('Team Name')

def train_on_tk(v2u):
    winner_vars = []
    loser_vars = []
    for v in v2u:
        if v != 'Team Name':
            winner_vars.append(v + ' Winner')
            loser_vars.append(v + ' Loser')
    

    # Load each training game into a np array that will be the
    # training data set
    Xs = []    # input vars
    ys = []    # output var
    for idx, row in train_raw_games.iterrows():
        w_v = np.array(row[winner_vars])
        l_v = np.array(row[loser_vars])
        Xs.append(np.array([w_v, l_v]).flatten())
        ys.append(np.array([row['Winner Points'] - row['Loser points']]))
        Xs.append(np.array([l_v, w_v]).flatten())
        ys.append(np.array([row['Loser points'] - row['Winner Points']]))

    # Convert lists to np array
    X = np.array(Xs)
    y = np.array(ys)

    # Load each training game into a np array that will be the
    # validation data set
    Xvs = []
    yvs = []
    for idx, row in val_raw_games.iterrows():
        winner = row['Winner']
        loser = row['Loser']

        try:
            w_v = np.array(row[winner_vars])
            l_v = np.array(row[loser_vars])
            Xvs.append(np.array([w_v, l_v]).flatten())
            yvs.append(np.array([row['Winner Points'] - row['Loser points']]))
            Xvs.append(np.array([l_v, w_v]).flatten())
            yvs.append(np.array([row['Loser points'] - row['Winner Points']]))
        except:
            print(winner)
            print(loser)

    Xv = np.array(Xvs)
    yv = np.array(yvs)

    # Train a Gaussian Process Regression model
    RBF_LENGTH_SCALE = 54.5618245614035
    RBF_KERNEL_SCALE = 18.538748538011692
    NOISE_LENGTH_SCALE = 84.34343436464646
    kernel = RBF_KERNEL_SCALE * RBF(RBF_LENGTH_SCALE) + WhiteKernel(NOISE_LENGTH_SCALE)
    gp = GaussianProcessRegressor(kernel,
                                n_restarts_optimizer=10,
                                normalize_y=True)
    gp.fit(X, y)
    rep = gp.score(Xv, yv)

    # Train the linear model on the same data
    lm = linear_model.LinearRegression(fit_intercept=False, normalize=True)
    lm.fit(X, y)
    lm.score(Xv, yv)

    gp_y_pred = []
    lm_y_pred = []
    gp_sd = []
    tys = []
    for idx, row in val_raw_games.iterrows():
        winner = row['Winner']
        loser = row['Loser']

        try:
            w_v = np.array(row[winner_vars])
            l_v = np.array(row[loser_vars])
            Xvs.append(np.array([w_v, l_v]).flatten())
            yvs.append(np.array([row['Winner Points'] - row['Loser points']]))

            gp_y_pred_, gp_sd_ = gp.predict(np.array([w_v, l_v]).flatten().reshape(1, -1), return_std=True)
            gp_y_pred.append(gp_y_pred_)
            gp_sd.append(gp_sd_)
            lm_y_pred.append(lm.predict(np.array([w_v, l_v]).flatten().reshape(1, -1)))
            tys.append(row['Winner Points'] - row['Loser points'])
        except:
            print(winner)
            print(loser)

    gp_y_pred=np.array(gp_y_pred).flatten()
    lm_y_pred=np.array(lm_y_pred).flatten()
    gp_sd = np.array(gp_sd).flatten()
    tys = np.array(tys).flatten()

    correct = 0
    for i in range(len(tys)):
        if tys[i] >= 0:
            if lm_y_pred[i] >= 0:
                correct += 1
        else:
            if lm_y_pred[i] < 0:
                correct += 1

    correct = 0
    for i in range(len(tys)):
        if tys[i] >= 0:
            if gp_y_pred[i] >= 0:
                correct += 1
        else:
            if gp_y_pred[i] < 0:
                correct += 1
                
    y_pred_probs = 1 - norm.cdf(-gp_y_pred / gp_sd)
    ll = log_loss(np.array(tys > 0, dtype=np.int), np.array([1 - y_pred_probs, y_pred_probs]).T, labels=[0, 1])

    return ll

In [4]:
max_ss = None
max_ll = 1
with open('mpi_log.txt', 'a') as fi:
    for i in range(3, 6):
        for to_keep in set(itertools.combinations(VARS, i)):
            ll = train_on_tk(['Team Name'] + list(to_keep))
            print(ll)
            if ll < max_ll:
                max_ll = ll
                max_ss = to_keep
                fi.write(f'{to_keep}: {ll}\n')

print('Best Subset: ' + str(max_ss))
print('Best Log Loss: ' + str(max_ll))

0.6303202167916506
0.5931508023441718
0.5659038310274184
0.5110208925013072
0.5152674532378629
0.6843041947666939
0.5416561398150095
0.6845808436901164
0.5240296363751584
0.6305903817984065
0.5531330433956286
0.5841311255104819
0.6858192515418404
0.6932048196141339
0.5868904936596827
0.579896418523526
0.5256645729693691
0.5975617605601745
0.5863971231595534
0.584282129152777
0.5340607265177956
0.5947149816527013
0.6932041501724785
0.6413606829285663
0.5755722746985203
0.5467500598679941
0.6908982340673123
0.5659425226134224
0.5629725036709591
0.6489662213745786
0.5344370443567942
0.5511006913408631
0.5198971506435884
0.5795607048669795
0.5199376991789513
0.5151920873479381
0.6919333802130478
0.6844260131807264
0.5211225528316468
0.5870170035298952
0.7003376644525673
0.593746280657393
0.5236335511060322
0.5928826658239451
0.5775129216425661
0.7000600665503288
0.5940716959122571
0.5793813371826195
0.6931471805599455
0.5489902460991316
0.6929517868787489
0.5485269940275064
0.5287763704387

0.5213859840164112
0.5929941970087624
0.5965465810559453
0.6849421628577377
0.5707992398058233
0.6938665893545584
0.5895977667980435
0.6874983573926033
0.6214331996283691
0.6206341641503317
0.5207597237017777
0.6931471805599455
0.5948997896402833
0.5387273558537441
0.6060795544570375
0.5204232468271526
0.6931471805599455
0.5471050344905534
0.5984115004971324
0.6041398719443868
0.6931471805599455
0.5154064088133866
0.5578085162667281
0.55707658691519
0.6931471978634575
0.5399427116859875
0.6671103582997312
0.521844039294595
0.5241812921901401
0.7001015768120515
0.524259131803169
0.620942220336972
0.6997121487033662
0.5629824677205113
0.6941554689555948
0.6283970433712246
0.6931471805599455
0.575570755817682
0.6944295764099008
0.6486453536656723
0.6421288731038005
0.6930871102955162
0.6113655357748109
0.6927068665588416
0.5490826044089429
0.5668656985790128
0.5627942136569772
0.630327060734428
0.6299760657092365
0.5391811925315342
0.5241943913949498
0.525086775589653
0.5506236603104308
0

0.6337956516377088
0.6066052067741815
0.6119994769638034
0.5489224205060012
0.5792687987125154
0.5220854456932662
0.5256588433966584
0.5345088715071108
0.572543970144887
0.5938366794095317
0.5848233970233417
0.6307712451078565
0.5929755277168286
0.7524892983731617
0.5556516483188717
0.5941137350230629
0.6931471593424212
0.5917299653600919
0.5598062486697966
0.5536630437921383
0.6075447989533727
0.5465965459129989
0.6065696139812133
0.5798350906267994
0.5851433562731942
0.5881136039301913
0.5089207692116136
0.6931471804445966
0.6929324038999858
0.5686031895348037
0.5730392191124769
0.5391035276725776
0.5952341633334323
0.5802291753094329
0.576833215289336
0.5559295715942275
0.6931471805599455
0.5241809455298185
0.6305717518159323
0.5708438336048155
0.6405822410123443
0.5749407036463585
0.5730412388833006
0.5239033601919655
0.6915458635489657
0.5956436578201387
0.5240670662688615
0.5885040265421954
0.5889242506985743
0.620074239904985
0.5783962814036335
0.5435818451962402
0.5814327955779

0.6931471805599455
0.5909504799510717
0.5224004361556809
0.5795994436947126
0.600207190135334
0.5811974579681549
0.530809617689924
0.5761516271510402
0.7000784381066939
0.5287822615365071
0.693147973944923
0.630024809835353
0.6213690754941307
0.6248156701627324
0.5957678726483543
0.5579697529373491
0.6912697349783661
0.5530188561488896
0.5519323995484311
0.5801724865446718
0.6481202178956381
0.5490347600587684
0.5964133138625192
0.6916870532746152
0.5310784298509512
0.6121911331579855
0.6112469484588935
0.6441723660149031
0.5443764873449752
0.6412364961015886
0.5724766128392499
0.6845112634865428
0.5766163354637743
0.5559251314829298
0.5938391553542678
0.5296231691039142
0.5487604075534678
0.5946138998178223
0.6895282420522523
0.6821894765117935
0.7090580295979574
0.522051180083612
0.5753575390107302
0.6150169632339882
0.69342769743792
0.5887600247867086
0.591936896659745
0.5945760551173085
0.5370771557154158
0.5911077268529088
0.5215500429779365
0.630622767520121
0.5767455127833887
0.

0.5803300227621353
0.5997544654909281
0.5391047221350647
0.5082248208130654
0.5531517673991468
0.5275268287833644
0.667675934640693
0.5904007095428646
0.5239190987806044
0.5108002045255235
0.5532306068051055
0.6841525173100982
0.5629719268968745
0.5594901134351576
0.5681986118438264
0.5893100236315681
0.5344121492573427
0.5663089112938284
0.5703126038966081
0.6851089087748223
0.5748007303677325
0.556830235526932
0.6012878411994066
0.6931471805599455
0.5676744469258825
0.6265977937591898
0.5794461909013321
0.7547366741699727
0.5575425423114265
0.5628256326380505
0.5259340210111384
0.5622667431969727
0.6053665506518695
0.6931471805599455
0.5949841981076488
0.551384258890044
0.5942202378475255
0.5575413730766163
0.7150465716004104
0.5965171468817982
0.522084954903836
0.5632851168862846
0.5436226405177654
0.6930491587583562
0.5766977149253247
0.577316092125403
0.5467398966627708
0.5722544326158743
0.6931471805599455
0.5466510090990249
0.6956390681611981
0.5776389942681107
0.529758077200457

...

0.6987288434550899
0.6946608137398912
0.6948687729359178
0.5552843201881958
0.5337805350272186
0.5782124981788231
0.6150350605647269
0.5368386858231217
0.6964915408077501
0.5677830091102797
0.6999165548797718
0.6813959812351119
0.5675730270541466
0.5204177823652825
0.5941079312338956
0.6244639702927818
0.6893274595861089
0.604168970545878
0.5780534759278675
0.5809270927256099
0.5228248720147717
0.6986560300128399
0.5239191901953127
0.5233270968781408
0.5767038742122849
0.6293069612162123
0.5612196304515561
0.6905531142378359
0.6207468140213594
0.5505854115418033
0.6422114509525315
0.5766107114132845
0.6903887616775037
0.5530813606718358
0.5773789558971695
0.640059174536926
0.5717153650800725
0.5788759743520792
0.6965209869674156
0.6981166401302179
0.5663560490973848
0.5375480826303809
0.570979294567181
0.596789146012982
0.5523421118923314
0.5793620727766758
0.578293032749442
0.5913200708245897
0.5238499786214177
0.5147294994548088
0.5570612118467693
0.5479277599491833
0.573443356401318

0.5091013763521488
0.5327714104299414
0.604634403272364
0.5943669222521605
0.5298843166093674
0.5474799223910798
0.6878648477475049
0.5442094895027585
0.5464640335289863
0.5495098125439966
0.7002904140902455
0.6880357145287359
0.5835152262336023
0.5658636834544033
0.684961430866753
0.5314820092923983
0.5791555166121688
0.6931471805599455
0.5524695211147932
0.5760707059358061
0.6416727650176312
0.5414806079850523
0.6909433426786039
0.5439076060828026
0.6063050280466038
0.5401424460806582
0.5870884685862064
0.5504722139020994
0.5316925149372731
0.5419430338628646
0.5825961508402341
0.5741907142727629
0.5712913656568888
0.6890127861688431
0.5771858909677868
0.6830445066307472
0.569063180632323
0.6845897794615605
0.5917845659626191
0.5477990396363543
0.6029394168538227
0.5940194271153375
0.5520905359432027
0.5713964538291679
0.5414838385142227
0.5761828470381265
0.5202652636506796
0.5802020071948267
0.6993587816446265
0.6666992109710739
0.5562657422268381
0.5057826304849421
0.6955275546171

0.5868916629624978
0.5641741414812741
0.6822092207366834
0.6910716717331568
0.5151785276018935
0.6956357798302886
0.6931473297381877
0.5551730754416361
0.5963161928394147
0.5158534862243845
0.5520423304796073
0.6894318518021837
0.5776351278051585
0.5722093861996645
0.5735834822205981
0.6906575544298398
0.687911691139635
0.5465393959063379
0.5796287781329175
0.5600012223856181
0.5530114736800731
0.5946164443094285
0.6931471805599455
0.5768728057066321
0.6829895629797942
0.5860957078906115
0.6206588204593007
0.570842796720972
0.5609487356704148
0.695006586310664
0.5869078721241662
0.5607150020200146
0.5349043750238658
0.5758188217031515
0.5343015142776432
0.544497273768261
0.6799588945876169
0.5701036381040779
0.6391274343981794
0.5176977970040361
0.5120513651433422
0.5559249968659677
0.6052428574163269
0.5048882975658668
0.5782494555590453
0.5884221263055541
0.5296939767228563
0.596406677354637
0.5598825140104902
0.5694110592435958
0.5634345121411745
0.5928563665903809
0.688301121447782

0.56719305004145
0.6422167821786569
0.624489428206988
0.6956316775274101
0.6897090743731898
0.687156741738842
0.5192323207006891
0.5981889507570296
0.628710542112806
0.5947542779644773
0.6933980108946745
0.6931471805599455
0.5652635151584978
0.5433129400503153
0.6128666952423245
0.5343922419376702
0.5626362027612005
0.5633556010672279
0.6853209088222351
0.6928670720227726
0.5214029629453438
0.5195260039831052
0.5755871959727038
0.7083867262707849
0.6889027045040171
0.532964219768064
0.5025082264057462
0.5826554425734741
0.5635514424291883
0.6944555730858609
0.5508715656864829
0.5751020970131178
0.6944309341690025
0.5865300990353436
0.572543099582559
0.5301600561091744
0.5226405390144613
0.7000789257207288
0.5297706478793517
0.5736695552208982
0.5671845200168101
0.5591338657385357
0.5671899985537834
0.5793631801158464
0.5724247129277088
0.6322527520577003
0.6101704724037298
0.56513374158368
0.5764913837627006
0.5254422499929663
0.5924535109470772
0.6878042414680078
0.5827584916081957
0.

0.6899890362387294
0.6931549748153888
0.5985432492041927
0.5312296883345142
0.5692452683599285
0.693716290007479
0.5772156951929152
0.5728097182436359
0.5002260964068631
0.6929902865245356
0.5593708303452926
0.561283685196941
0.6913929926184361
0.5774238685821859
0.528966268839491
0.5210034317116834
0.6931471805599455
0.6084730186005044
0.6302338558017413
0.6929329085931801
0.5840533940118758
0.5337124161275456
0.5183933367876877
0.6903877363372039
0.5500929386700067
0.6349314147742187
0.5570783341239935
0.5380957045677275
0.5639136245528055
0.5477087397678866
0.5107978161532089
0.5769173577771572
0.6217762779298385
0.500173490545308
0.6846786992688985
0.6930887077151896
0.5319544790993836
0.5376805702556485
0.5941168724129418
0.6931471804583369
0.531256849114036
0.5575172923779611
0.5780690755964134
0.6892042212058305
0.5899474088062004
0.5212278709552995
0.5791072001078151
0.5956896925096956
0.5352590240856134
0.5387489830152807
0.6920129942846569
0.5252007215597224
0.593499995576103